<center><h1 style="font-size:40px;">Exercise II: Classification</h1></center>

---

For this exercise you are given two classification problems with a fixed training-, validation- and test dataset. A spiral dataset and a japanese vowels dataset.

The **task** is to do model selection, coming up with your optimal MLP architecture together with the hyperparameters. For this part no code for the configuration or training are given. Please implement this based on the previous parts. A small evaluation section is provided to observe the confusion matrix for the prediction vs target.

# Data
## Japanese vowels dataset
This data set is taken from the UCI Machine Learning Repository [https://archive.ics.uci.edu/ml/datasets/Japanese+Vowels]. In short, nine male speakers uttered two Japanese vowels /ae/ successively. For each utterance, a discrete times series was produced where each time point consists of 12 (LPC cepstrum) coefficients. The length of each time series was between 7-29. 
Here we treat each point of the time series as a feature (12 inputs). In total we have 9961
data points which then has been divided into 4274 for training, 2275 for validation and 3412 for test. The original data files are provided as *ae.train* and *ae.test*. The task is now based on a single sample value of one of the speakers, determine which speaker it was. This is, in summary, a 9-class classification problem with 12 input values for each case.

## Spiral data
This is the "famous" spiral dataset that consists of two 2-D spirals, one for each class. The perfect classification boundary is also a spiral. The cell "PlotData" will plot this dataset.

# Code

The following code allows us to edit imported files without restarting the kernel for the notebook

In [1]:
%load_ext autoreload
%autoreload 2

# Hacky solution to access the global utils package
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('')))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl

from config import LabConfig
from dataset import MLPData
from utils.model import Model
from utils.progressbar import LitProgressBar
from utils.model import Model
from torch.utils.data import TensorDataset, DataLoader
import torchmetrics
from utils import (
    plot,
    progressbar
) 

In [3]:
cfg = LabConfig()

## Task 1
**TODO:** Present an MLP for the Japanese vowels dataset with associated hyperparameters that maximizes the validation performance and give the test performance you obtained. \
**TODO:** Motivate the choice of parameters and implementation. 

**Hint 2:** This problem is a 9-class classification problem, meaning that you should use a specific output activation function (*out_act_fun*) and a specific loss/error function (*cost_fun*).

In [4]:
# Load dataset - Vowels
from utils.utils import onehot2int
def numpy2Dataloader(x,y, batch_size=25, num_workers=10,**kwargs):
    return DataLoader(
        TensorDataset(
            torch.from_numpy(x).float(), 
            torch.from_numpy(np.argmax(y,axis=1)).long()
        ),
        batch_size=batch_size,
        num_workers=num_workers,
        **kwargs
    )

x_train, y_train, x_val, y_val, x_test, y_test = MLPData.vowels(file_name_train=cfg.ae_train, file_name_test=cfg.ae_test)

train_loader = numpy2Dataloader(x_train,y_train)
val_loader =  numpy2Dataloader(x_val,y_val)
test_loader =  numpy2Dataloader(x_test,y_test)

num_classes = 9
print(f'|{"Type":10} | {"Input size":10} | {"Target size":10}|')
print(f'|{"-"*11}|{"-"*12}|{"-"*12}|')
print(f'|{"train":10} | {str(x_train.shape):10} | {str(y_train.shape):10} |')
print(f'|{"val":8}   | {str(x_val.shape):10} | {str(y_val.shape):10} |')
print(f'|{"test":9}  | {str(x_test.shape):10} | {str(y_test.shape):10} |')

|Type       | Input size | Target size|
|-----------|------------|------------|
|train      | (4274, 12) | (4274, 9)  |
|val        | (2275, 12) | (2275, 9)  |
|test       | (3412, 12) | (3412, 9)  |


In [ ]:
# TODO - Create model 
import torch.nn.functional as F

class MLP(torch.nn.Module):
    def __init__(self, 
                inp_dim=None,         
                hidden_nodes=1,                      # number of nodes in hidden layer
                hidden_layers=1,                     # number of hidden layers in the network
                num_out=None,
                **kwargs
            ):
        super(MLP, self).__init__()
        model = []
        
        model.append(torch.nn.Linear(inp_dim, hidden_nodes)) # Input layer
        for layer_i in range(hidden_layers - 1):
            model.append(torch.nn.ReLU())
            model.append(torch.nn.Linear(hidden_nodes, hidden_nodes))
            
        model.append(torch.nn.ReLU())
        model.append(torch.nn.Linear(hidden_nodes, num_out))   
        
        self.seqModel = torch.nn.Sequential(*model)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        output = self.seqModel(x)
        return output

# TODO - Setup configurations

config = {
    'max_epochs':100,
    'model_params':{
        'inp_dim':x_train.shape[1],         
        'hidden_nodes':100,   # activation functions for the hidden layer,
        'hidden_layers':1,
        'num_out':y_train.shape[1] # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
    },
    'criterion':torch.nn.CrossEntropyLoss(), # error function
    'optimizer':{
        "type":torch.optim.Adam,
        "args":{
            "lr":0.005,
            'weight_decay':0.002
            
        }
    }
}

# TODO - Run model

def model_eval(config_, train_loader_, val_loader_):
    model = Model(MLP(**config["model_params"]),**config)
    trainer = pl.Trainer(
            max_epochs=config['max_epochs'], 
            gpus=cfg.GPU,
            logger=pl.loggers.TensorBoardLogger(save_dir=cfg.TENSORBORD_DIR),
            callbacks=[pl.callbacks.progress.TQDMProgressBar(1)],
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10,   
        )

    trainer.fit(
        model, 
        train_dataloaders=train_loader,
        val_dataloaders=val_loader,
    );

for hidden_nodes in [1, 10, 20, 50, 100, 400, 1000]:
    print('hidden_nodes: ' + str(hidden_nodes))
    for l2 in [0.001, 0.005, 0.01, 0.05, 0.09]:
        print('l2: ' + str(l2))
        config = {
            'max_epochs':100,
            'model_params':{
                'inp_dim':x_train.shape[1],         
                'hidden_nodes':hidden_nodes,   # activation functions for the hidden layer,
                'hidden_layers':1,
                'num_out':y_train.shape[1] # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
            },
            'criterion':torch.nn.CrossEntropyLoss(), # error function
            'optimizer':{
                "type":torch.optim.Adam,
                "args":{
                    "lr":0.005,
                    'weight_decay':l2

                }
            }
        }
        model_eval(config, train_loader, val_loader)
    print('-------------------')
        
# TODO - Validate performance

predictions = torch.argmax(torch.nn.functional.softmax(model(torch.from_numpy(x_test).float())),dim=1)
target = torch.argmax(torch.from_numpy(y_test), axis=1)

confuTst = torchmetrics.functional.confusion_matrix(predictions.detach().cpu(),target.int().detach().cpu(), cfg.AE_NUM_CLASSES)

plot.confusion_matrix(cm = confuTst.numpy(), 
                      normalize = False,
                      target_names = cfg.AE_CLASSES,
                      title = "Confusion Matrix: Test data")

/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:167: LightningDeprecationWarning: Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


hidden_nodes: 1
l2: 0.001
Epoch 0:  65%|██████████████████████████████████████████████████████████████████████████████████████▊                                              | 171/262 [00:01<00:00, 170.17it/s, loss=2.93, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 1:  66%|███████████████████████████████████████████████████████████████████████████████████████▎                                             | 172/262 [00:00<00:00, 173.04it/s, loss=2.78, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 2:  66%|███████████████████████████████████████████████████████████████████████████████████████▎                                             | 172/262 [00:00<00:00, 181.25it/s, loss=2.67, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 3:  66%|███████████████████████████████████████████████████████████████████████████████████████▎                                             | 172/262 [00:01<00:00, 149.57it/s, loss=2.82, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 4:  66%|████████████████

Epoch 8:  66%|███████████████████████████████████████████████████████████████████████████████████████▎                                             | 172/262 [00:01<00:00, 159.66it/s, loss=2.49, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 9:  66%|███████████████████████████████████████████████████████████████████████████████████████▎                                             | 172/262 [00:00<00:00, 184.81it/s, loss=2.45, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 10:  66%|███████████████████████████████████████████████████████████████████████████████████████▎                                             | 172/262 [00:01<00:00, 157.10it/s, loss=2.4, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 11:  66%|██████████████████████████████████████████████████████████████████████████████████████▋                                             | 172/262 [00:01<00:00, 168.16it/s, loss=2.36, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 12:  66%|█████████████████████████████████████████

Epoch 16:  66%|██████████████████████████████████████████████████████████████████████████████████████▋                                             | 172/262 [00:01<00:00, 155.48it/s, loss=2.13, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 17:  65%|██████████████████████████████████████████████████████████████████████████████████████▊                                              | 171/262 [00:01<00:00, 168.02it/s, loss=2.1, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 18:  65%|██████████████████████████████████████████████████████████████████████████████████████▏                                             | 171/262 [00:00<00:00, 174.23it/s, loss=2.06, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 19:  65%|██████████████████████████████████████████████████████████████████████████████████████▏                                             | 171/262 [00:01<00:00, 170.52it/s, loss=2.04, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 20:  65%|█████████████████████████████████████████

Validating: 0it [00:00, ?it/s]
Epoch 25:  65%|██████████████████████████████████████████████████████████████████████████████████████▏                                             | 171/262 [00:00<00:00, 201.54it/s, loss=1.91, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 26:  65%|██████████████████████████████████████████████████████████████████████████████████████▊                                              | 171/262 [00:01<00:00, 155.15it/s, loss=1.9, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 27:  65%|██████████████████████████████████████████████████████████████████████████████████████▏                                             | 171/262 [00:01<00:00, 157.76it/s, loss=1.89, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 28:  65%|██████████████████████████████████████████████████████████████████████████████████████▏                                             | 171/262 [00:01<00:00, 150.00it/s, loss=1.88, v_num=127]
Validating: 0it [00:00, ?it/s]
Epoch 29:  65%|██████████

...

  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/IPython/core/ultratb.py", line 281, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 1541, in getinnerframes
    frameinfo = (tb.tb_frame,) + getframeinfo(tb, context)
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 1499, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 709, in getsourcefile
    if getattr(getmodule(object, filename), '__loader__', None) is not None:
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 755, in getmodule
    os.path.realpath(f)] = module.__name__
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/posixpath.py", line 392, in realpath
    path, ok = _joinr

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2373997/4036636539.py", line 97, in <module>
    predictions = torch.argmax(torch.nn.functional.softmax(model(torch.from_numpy(x_test).float())),dim=1)
NameError: name 'model' is not defined
Traceback (most recent call last):
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2373997/4036636539.py", line 97, in <module>
    predictions = torch.argmax(torch.nn.functional.softmax(model(torch.from_numpy(x_test).float())),dim=1)
NameError: name 'model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3364, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3461, in run_code
    self.showtraceback(running_compiled_code=True)
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2066, in showtraceback
    s

  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 1541, in getinnerframes
    frameinfo = (tb.tb_frame,) + getframeinfo(tb, context)
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 1499, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 709, in getsourcefile
    if getattr(getmodule(object, filename), '__loader__', None) is not None:
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 752, in getmodule
    f = getabsfile(module)
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 721, in getabsfile
    _filename = getsourcefile(object) or getfile(object)
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/inspect.py", line 706, in getsourcefile
    if os.path.exists(filename):
  File "/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python

## Task 2
For this last exercise the task is to train a binary classifier for the spiral problem. The aim is to get *zero* classification error for the training data (there is no test data) with as small as possible model, in terms of the number of trainable weights. Also plot the boundary to see if it resembles a spriral. To pass this question you should at least try!

**TODO:** Train a classifier for the spiral problem with the aim of zero classification error with as small as possible model. 

**TODO:** Motivate the choice of parameters and implementation.

In [ ]:
# Load dataset
def numpy2Dataloader(x,y, batch_size=25, num_workers=10,**kwargs):
    return DataLoader(
        TensorDataset(
            torch.from_numpy(x).float(), 
            torch.from_numpy(y).long()
        ),
        batch_size=batch_size,
        num_workers=num_workers,
        **kwargs
    )
x_train, y_train = MLPData.spiral(cfg.spiral_path)
train_loader = numpy2Dataloader(x_train,y_train)

## Implement the TODO's

In [ ]:
# TODO - Create model 

class MLP(torch.nn.Module):
    def __init__(self, 
                inp_dim=None,         
                hidden_nodes=1,         # number of nodes in hidden layer
                hidden_layers=1,        # number of hidden layers in the network
                num_out=None,
                **kwargs
            ):
        super(MLP, self).__init__()
        model = []
        
        model.append(torch.nn.Linear(inp_dim, hidden_nodes)) # Input layer
        for layer_i in range(hidden_layers - 1):
            model.append(torch.nn.ReLU())
            model.append(torch.nn.Linear(hidden_nodes, hidden_nodes))
            
        model.append(torch.nn.ReLU())
        model.append(torch.nn.Linear(hidden_nodes, num_out))   
        
        self.seqModel = torch.nn.Sequential(*model)
        
    def forward(self, x):
        output = self.seqModel(x)
        return output

# TODO - Setup configurations

config = {
    'max_epochs':500,
    'model_params':{
        'inp_dim':x_train.shape[1],         
        'hidden_nodes':16,   # activation functions for the hidden layer,
        'hidden_layers':4,
        'num_out': 1 # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
    },
    'criterion':torch.nn.BCEWithLogitsLoss(), # error function
    'optimizer':{
        "type":torch.optim.Adam,
        "args":{
            "lr":0.005,
            
        }
    }
}

# TODO - Run model

model = Model(MLP(**config["model_params"]),**config)
trainer = pl.Trainer(
        max_epochs=config['max_epochs'], 
        gpus=1,
        logger=pl.loggers.TensorBoardLogger(save_dir=cfg.TENSORBORD_DIR),
        callbacks=[pl.callbacks.progress.TQDMProgressBar(1)],
        weights_summary=None, # Can be None, top or full
        num_sanity_val_steps=10,
    )

trainer.fit(
    model, 
    train_dataloaders=train_loader
);

# TODO - Validate performance

plot.stats_class(x_train, y_train, 'Training', model)
plot.decision_bondary(trainer.lightning_module, x_train, y_train.reshape(-1, ))

## Example of evaluation
Run the testset and evaluate the performance of the model.

In [ ]:
# Move to correct device!
trainer.test(model, test_dataloaders=test_loader)

In [ ]:
trainer.logged_metrics

### Confusion matrix 

In [ ]:
predictions = torch.argmax(torch.nn.functional.softmax(model(torch.from_numpy(x_train).float())),dim=1)
target = torch.argmax(torch.from_numpy(y_train), axis=1)

confuTst = torchmetrics.functional.confusion_matrix(predictions.detach().cpu(),target.int().detach().cpu(), cfg.AE_NUM_CLASSES)

plot.confusion_matrix(cm = confuTst.numpy(), 
                      normalize = False,
                      target_names = cfg.AE_CLASSES,
                      title = "Confusion Matrix: Test data")